In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'gini', 'max_depth': 12}
----------
[[11582  4452]
 [  583 15370]]
----------
              precision    recall  f1-score   support

           0       0.95      0.72      0.82     16034
           1       0.78      0.96      0.86     15953

    accuracy                           0.84     31987
   macro avg       0.86      0.84      0.84     31987
weighted avg       0.86      0.84      0.84     31987

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.27925217, 0.3460747 , 0.37998343, 0.46226251, 0.41289604,
        0.45578063, 0.47822011, 0.53905833, 0.5226022 , 0.58344054,
        0.62881875, 0.70212102, 0.72107279, 0.68716097, 0.82379603,
        0.38147926, 0.42386532, 0.49019015, 0.60139191, 0.64677227,
        0.67918432, 0.75348532, 0.77841842, 0.82280004, 0.90557647,
        0.88962054, 0.96092892, 1.01628268, 0.9639219 , 0.97292829]),
 'std_fit_time': array([2.99131870e-03, 2.39356756e-02, 9.97304916e-04, 1.54589415e-02,
        3.98981571e-03, 9.97292995e-03, 1.74535513e-02, 4.48834896e-03,
        8.97705555e-03, 3.59053612e-02, 3.74000072e-02, 1.19209290e-06,
        1.99452639e-02, 1.39629841e-02, 3.69012356e-02, 2.54321098e-02,
        5.98454475e-03, 8.47804546e-03, 4.98688221e-03, 2.84243822e-02,
        3.29116583e-02, 1.94486380e-02, 8.47756863e-03, 2.59310007e-02,
        4.38804626e-02, 1.99532509e-03, 1.44629478e-02, 8.97538662e-03,
        1.04717016e-02, 6.33585453e-02]),
 'mean_scor